In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp310-cp310-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached https://download.pytorch.org/whl/nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl (188.7 MB)
  Using cached https://download.pytorch.org/whl/triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: sympy━━━━━━━━━━━━━━━━━━━ 0/5 [triton]
    Found existing installation: sympy 1.14.0 0/5 [triton]
    Uninstalling sympy-1.14.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [sympy]
      Successfully 

In [1]:
pip install git+https://github.com/bowang-lab/scGPT.git


  Cloning https://github.com/bowang-lab/scGPT.git to /tmp/pip-req-build-c6afkzw5
  Running command git clone --filter=blob:none --quiet https://github.com/bowang-lab/scGPT.git /tmp/pip-req-build-c6afkzw5
  Resolved https://github.com/bowang-lab/scGPT.git to commit 549178361f2e40fe8179a836af37b980e30e2b02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --force-reinstall torch==2.2.2+cu121 torchvision==0.17.2+cu121 torchaudio==2.2.2+cu121 torchtext==0.17.2 --extra-index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.2%2Bcu121-cp310-cp310-linux_x86_64.whl (757.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.17.2%2Bcu121-cp310-cp310-linux_x86_64.whl (7.0 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.2.2%2Bcu121-cp310-cp310-linux_x86_64.whl (3.4 MB)
  Using cached torchtext-0.17.2-cp310-cp310-manylinux1_x86_64.whl.metadata (7.9 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-

In [3]:
pip install "numpy<2" --force-reinstall


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import numpy as np
import scanpy as sc
import torch
from pathlib import Path
import os
import time
import pandas as pd

from scgpt.model import TransformerModel
from scgpt.tokenizer import GeneVocab
from scgpt.utils import load_pretrained
from scgpt.data_collator import DataCollator
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm


# -------------------------
# Utility: batch embedding
# -------------------------
def get_batch_cell_embeddings(
    adata,
    model,
    vocab,
    model_configs,
    gene_ids,
    max_length=1200,
    batch_size=64,
) -> np.ndarray:
    """
    Compute scGPT embeddings for all cells in AnnData.
    """

    # --- convert X to numpy array ---
    count_matrix = adata.X
    if not isinstance(count_matrix, np.ndarray):
        try:
            count_matrix = count_matrix.toarray()
        except Exception:
            count_matrix = np.asarray(count_matrix)

    # --- dataset class ---
    class Dataset(torch.utils.data.Dataset):
        def __init__(self, count_matrix, gene_ids):
            self.count_matrix = count_matrix
            self.gene_ids = gene_ids

        def __len__(self):
            return len(self.count_matrix)

        def __getitem__(self, idx):
            row = self.count_matrix[idx]
            nonzero_idx = np.nonzero(row)[0]
            values = row[nonzero_idx].astype(np.float32)
            genes = self.gene_ids[nonzero_idx]

            # prepend <cls>
            genes = np.insert(genes, 0, vocab["<cls>"])
            values = np.insert(values, 0, model_configs["pad_value"]).astype(np.float32)

            return {
                "id": idx,
                "genes": torch.tensor(genes, dtype=torch.long),
                "expressions": torch.tensor(values, dtype=torch.float32),
            }

    # --- data loader ---
    dataset = Dataset(count_matrix, gene_ids)
    collator = DataCollator(
        do_padding=True,
        pad_token_id=vocab[model_configs["pad_token"]],
        pad_value=model_configs["pad_value"],
        do_mlm=False,
        do_binning=False,
        max_length=max_length,
        sampling=True,
        keep_first_n_tokens=1,
    )
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=SequentialSampler(dataset),
        collate_fn=collator,
        drop_last=False,
        num_workers=0,  
        pin_memory=True,
    )

    # --- compute embeddings ---
    device = next(model.parameters()).device
    cell_embeddings = np.zeros((len(dataset), model_configs["embsize"]), dtype=np.float32)

    with torch.no_grad():
        count = 0
        for data_dict in tqdm(loader, desc="Embedding cells"):
            input_gene_ids = data_dict["gene"].to(device)
            src_key_padding_mask = input_gene_ids.eq(vocab[model_configs["pad_token"]])
            emb = model._encode(
                input_gene_ids,
                data_dict["expr"].to(device),
                src_key_padding_mask=src_key_padding_mask,
            )
            emb = emb[:, 0, :]  # take <cls> embedding
            emb = emb.cpu().numpy()
            cell_embeddings[count : count + len(emb)] = emb
            count += len(emb)

    # normalize embeddings
    cell_embeddings = cell_embeddings / np.linalg.norm(
        cell_embeddings, axis=1, keepdims=True
    )
    return cell_embeddings


# -------------------------
# Main embedding function
# -------------------------
def embed_with_scgpt(
    adata_or_file,
    model_dir: str,
    gene_col: str = "feature_name",
    max_length=1200,
    batch_size=64,
    device="cuda",
):
    # load AnnData
    if isinstance(adata_or_file, str):
        adata = sc.read_h5ad(adata_or_file)
    else:
        adata = adata_or_file

    model_dir = Path(model_dir)
    vocab_file = model_dir / "vocab.json"
    model_config_file = model_dir / "args.json"
    model_file = model_dir / "best_model.pt"

    # load vocab
    vocab = GeneVocab.from_file(vocab_file)
    for tok in ["<pad>", "<cls>", "<eoc>"]:
        if tok not in vocab.get_stoi():
            vocab.append_token(tok)

    # match genes
    if gene_col == "index":
        gene_list = adata.var.index.tolist()
    else:
        gene_list = adata.var[gene_col].tolist()

    adata.var["id_in_vocab"] = [
        vocab[g] if g in vocab.get_stoi() else -1 for g in gene_list
    ]

    adata = adata[:, adata.var["id_in_vocab"] >= 0]
    gene_ids = np.array(adata.var["id_in_vocab"])

    # load model configs
    with open(model_config_file, "r") as f:
        model_configs = json.load(f)

    model = TransformerModel(
        ntoken=len(vocab),
        d_model=model_configs["embsize"],
        nhead=model_configs["nheads"],
        d_hid=model_configs["d_hid"],
        nlayers=model_configs["nlayers"],
        nlayers_cls=model_configs["n_layers_cls"],
        n_cls=1,
        vocab=vocab,
        dropout=model_configs["dropout"],
        pad_token=model_configs["pad_token"],
        pad_value=model_configs["pad_value"],
        do_mvc=True,
        do_dab=False,
        use_batch_labels=False,
        domain_spec_batchnorm=False,
        explicit_zero_prob=False,
        use_fast_transformer=True,
        fast_transformer_backend="flash",
        pre_norm=False,
    )
    ckpt = torch.load(model_file, map_location="cpu")
    load_pretrained(model, ckpt, verbose=False)
    model.to(device)
    model.eval()

    # compute embeddings
    cell_embeddings = get_batch_cell_embeddings(
        adata,
        model=model,
        vocab=vocab,
        model_configs=model_configs,
        gene_ids=gene_ids,
        max_length=max_length,
        batch_size=batch_size,
    )
    adata.obsm["X_scGPT"] = cell_embeddings
    return adata


/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/opt/venv/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Input directory containing your .h5ad samples
input_dir = "/workspace/Projects/FM/Final data/H5AD/Visium"

# Output directory for embedded files
output_dir = "/workspace/Projects/FM/Final data/Visium_embeddings_scgpt"
os.makedirs(output_dir, exist_ok=True)

# Model directory (with vocab.json, args.json, best_model.pt)
model_dir = "/workspace/Projects/FM/Codes/scGPT_human-20250906T175545Z-1-001/scGPT_human"

# Collect all .h5ad files
h5ad_files = [f for f in os.listdir(input_dir) if f.endswith(".h5ad")]

In [6]:
summary_records = []


for f in h5ad_files:
    file_path = os.path.join(input_dir, f)
    print(f"\nProcessing {file_path}")


    adata = sc.read_h5ad(file_path)
    adata.var.index = adata.var["human_symbol"].astype(str)

    start_time = time.time()
    adata = embed_with_scgpt(
        adata,
        model_dir=model_dir,
        gene_col="index",   
        batch_size=8,
    )
    elapsed = time.time() - start_time

    shape = adata.obsm["X_scGPT"].shape
    print(f"Embedding shape: {shape}")
    print(f"Time taken: {elapsed:.2f} seconds")

    out_path = os.path.join(output_dir, f.replace(".h5ad", "_scgpt.h5ad"))
    adata.write(out_path)
    print(f" Saved embeddings → {out_path}")


    summary_records.append({
        "sample": f,
        "n_cells": shape[0],
        "embedding_dim": shape[1],
        "runtime_seconds": round(elapsed, 2),
        "status": "success"
    })


summary_df = pd.DataFrame(summary_records)
excel_path = os.path.join(output_dir, "embedding_runtime_summary.xlsx")
summary_df.to_excel(excel_path, index=False)

print(f"\n Summary saved to {excel_path}")


Processing /workspace/Projects/FM/Final data/H5AD/Visium/53430.h5ad


/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
Embedding cells:   0%|          | 0/534 [00:00<?, ?it/s]/opt/venv/jupyter/lib/python3.10/site-packages/torch/nn/modules/transformer.py:384: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
Embedding cells: 100%|██████████| 534/534 [00:23<00:00, 22.31it/s]
/tmp/ipykernel_766/3851143859.py:196: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings


Embedding shape: (4271, 512)
Time taken: 401.81 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/53430_scgpt.h5ad

Processing /workspace/Projects/FM/Final data/H5AD/Visium/26933.h5ad


/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
Embedding cells: 100%|██████████| 254/254 [00:11<00:00, 22.48it/s]
/tmp/ipykernel_766/3851143859.py:196: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings


Embedding shape: (2031, 512)
Time taken: 388.47 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/26933_scgpt.h5ad

Processing /workspace/Projects/FM/Final data/H5AD/Visium/26934.h5ad


...

Embedding shape: (2718, 512)
Time taken: 400.72 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/53431_scgpt.h5ad

Processing /workspace/Projects/FM/Final data/H5AD/Visium/53434.h5ad


/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
Embedding cells: 100%|██████████| 397/397 [00:17<00:00, 22.62it/s]
/tmp/ipykernel_766/3851143859.py:196: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings


Embedding shape: (3176, 512)
Time taken: 407.47 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/53434_scgpt.h5ad

Processing /workspace/Projects/FM/Final data/H5AD/Visium/53432.h5ad


/opt/venv/jupyter/lib/python3.10/site-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
Embedding cells: 100%|██████████| 422/422 [00:19<00:00, 22.17it/s]
/tmp/ipykernel_766/3851143859.py:196: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings


Embedding shape: (3376, 512)
Time taken: 405.65 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/53432_scgpt.h5ad

 Summary saved to /workspace/Projects/FM/Final data/Visium_embeddings_scgpt/embedding_runtime_summary.xlsx
